In [ ]:
import pandas as pd
import numpy as np

# Creating the dataframe to collect all the data.
blank_cols = np.arange(1,12,1)
fluor_conc = []
for i in range(0,11):
    fluor_conc.append(10/(2**i))
row_data = {
    'Column' : blank_cols,
    'Fluor_conc' : fluor_conc
}
blank_corrected = pd.DataFrame(row_data)


In [ ]:
#Importing data from the Excel sheets
dfs = {}
for i in range(1,16):
    dfs[f"df{i}"] = pd.read_excel(
        io = '/Users/wilsonporteus/Documents/GitHub/ssb_s5/Main Assesment/Data/bb'+str(i)+'.xlsx',
        #change this for your data file path
        header=0,
        sheet_name = 'End point', usecols="B:L",
        skiprows=13,
        #might need to change the usecols and skiprows arguments depending on the format of the microplate reader.
        nrows=9, engine="openpyxl"
        )
    avg = []
    for col in range(0,11):
        avg.append(
            np.average(dfs[f"df{i}"].iloc[:,col])
        )
    blank_corrected.insert(loc=i+1, column=f"df{i}_bc", value=avg)

log_df = np.log10(blank_corrected)

In [ ]:
#Determining the Fit (Gradient of Log-Log trendline) and R^2 of the trendline.
slopes = []
r2_vals = []
x = log_df["Fluor_conc"]

for col in range(2,17):
    y = log_df.iloc[:,col]
    m, b = np.polyfit(x, y, 1)
    y_pred = m * x + b
    ss_res = np.sum((y - y_pred)**2)
    ss_tot = np.sum((y - np.mean(y))**2)
    r2 = 1 - (ss_res / ss_tot)
    slopes.append(m)
    r2_vals.append(r2)

exp_number = np.arange(1,16,1)
final_data = {
    "Expirement Number" : exp_number,
    "Gradient" : slopes,
    "R2" : r2_vals
}
final_df = pd.DataFrame(final_data)
#Saves to an excel file in this directory
final_df.to_excel("BB_gradient.xlsx")